<a href="https://colab.research.google.com/github/Excalib3r/pinecone/blob/main/ph3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.7 MB/s eta 0:00:00


In [ ]:
import openai
import difflib
import random


openai.api_key = ''  # Replace with your OpenAI API key

# Define the questions and answers
questions_answers = [
    {
        'question': 'What is on-call time?',
        'answer': 'On-call time refers to the period when a non-exempt employee is required to be available in case they are needed to work.'
    },
    {
        'question': 'When is on-call time considered compensable?',
        'answer': 'On-call time is considered compensable if the employee is required to remain on the employer\'s premises or nearby, and cannot effectively use the time for their own purposes.'
    },
     {
        'question': 'How can employees determine if their on-call time is compensable?',
        'answer': 'If the response time and geographical restrictions are less restrictive and employees are able to engage in activities they normally do while not on-call, it is likely that the time spent on-call is not compensable. Employees can also contact Human Resources for clarification.'
    },
     {
        'question': ' Is all time spent working or responding to a call compensable?',
        'answer': 'Yes, all time spent working or responding to a call is compensable for non-exempt employees.'
    },
     {
        'question': 'Is travel time to and from the worksite compensable for on-call employees?',
        'answer': 'Yes, travel time to and from the worksite is compensable for on-call employees. This includes time spent driving and any time spent at the worksite.'
    },
    # Add more questions and answers here
]

# Function to get a ranking and missing information from the OpenAI API
system_template = """You are an expert in comparing user answers to the correctly defined answers.
You will be given text that contains many questions and answers related to those questions along with the users answer to the question
Identify any key points missing from the user's answer and list them. Also, rank the completeness and accuracy of the user's answer on a scale from 1 to 5.\n\nQ: {question}\nA: {answer}\nUser Answer: {user_answer}
Do not respond with information that isn't relevant to the question or the answer provided to you.
Do not make anything up."""

# ...

def get_ranking_and_missing_info(question, answer, user_answer):
    context = {
        "question": question,
        "answer": answer,
        "user_answer": user_answer,
    }

    # Formulate the system and user messages
    system_message = {
        "role": "system",
        "content": system_template.format(**context),
    }
    user_message = {
        "role": "user",
        "content": user_answer,
    }

    # The list of messages to be processed
    messages_to_process = [system_message, user_message]

    response = qa.run(messages_to_process)

    response_text = response['choices'][0]['text'].strip()

    return response_text




# Function to get a random question from the list
def get_random_question():
    return random.choice(questions_answers)

# Main loop for asking questions and getting user answers
def main():
    while True:
        question_dict = get_random_question()
        question = question_dict['question']
        answer = question_dict['answer']

        user_answer = input(f"\nQ: {question}\nYour Answer: ")

        result = get_ranking_and_missing_info(question, answer, user_answer)

        print(result)

        # Continue the loop or break based on your app's requirements
        choice = input("\nDo you want to continue? (y/n): ")
        if choice.lower() != 'y':
            break

# Call the main function
main()




Q:  Is all time spent working or responding to a call compensable?
Your Answer: it isnt


InvalidRequestError: ignored

changing prompt to openai function call so i can get structured data out of the llm

---



In [ ]:
pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.6 MB/s eta 0:00:00


In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import create_extraction_chain
import pinecone

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
OPENAI_API_KEY = '' # add your key here

In [ ]:
# Creating two versions of the model so I can swap between gpt3.5 and gpt4
llm3 = ChatOpenAI(temperature=0,
                  openai_api_key=OPENAI_API_KEY,
                  model_name="gpt-3.5-turbo-0613",
                  request_timeout = 180
                )

llm4 = ChatOpenAI(temperature=0,
                  openai_api_key=OPENAI_API_KEY,
                  model_name="gpt-4-0613",
                  request_timeout = 180
                 )

In [ ]:
transcript_paths = [
    './qna.txt',
]

with open('./qna.txt') as file:
    transcript = file.read()

In [ ]:
print(transcript[:100])

Dress code and uniform policy: Guidelines for appropriate attire and uniforms for employees.
Q: What


In [ ]:
# Load up your text splitter
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", " "], chunk_size=10000, chunk_overlap=2200)

# I'm only doing the first 23250 characters. This to save on costs. When you're doing your exercise you can remove this to let all the data through
transcript_subsection_characters = 23250

In [ ]:
pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.9 MB/s eta 0:00:00


In [ ]:
docs = text_splitter.create_documents([transcript[:transcript_subsection_characters]])
print (f"You have {len(docs)} docs. First doc is {llm3.get_num_tokens(docs[0].page_content)} tokens")

You have 2 docs. First doc is 2059 tokens


In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [ ]:
!pip3 install pinecone-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 24.1 MB/s eta 0:00:00


In [ ]:
PINECONE_API_KEY = ''
PINECONE_API_ENV = ''

In [ ]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)

In [ ]:
index_name = ""

docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name, namespace="qna")

In [ ]:
# The system instructions. Notice the 'context' placeholder down below. This is where our relevant docs will go.
# The 'question' in the human message below won't be a question per se, but rather a topic we want to get relevant information on
system_template = """You are an expert in comparing user answers to the correctly defined answers.
You will be given text that contains many questions and answers related to those questions along with the users answer to the question
Identify any key points missing from the user's answer and list them. Also, rank the completeness and accuracy of the user's answer on a scale from 1 to 5.\n\nQ: {question}\nA: {answer}\nUser Answer: {user_answer}
Do not respond with information that isn't relevant to the question or the answer provided to you.
Do not make anything up.
----------------
{context}"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}"),
]

# This will pull the two messages together and get them ready to be sent to the LLM through the retriever
CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)

In [ ]:
context = {
        "question": question,
        "answer": answer,
        "user_answer": user_answer,
    }

In [ ]:
# I'm using gpt4 for the increased reasoning power.
# I'm also setting k=4 so the number of relevant docs we get back is 4. This parameter should be tuned to your use case
qa = RetrievalQA.from_chain_type(llm=llm3,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(k=3),
                                 chain_type_kwargs = {
#                                      'verbose': True,
                                     'prompt': CHAT_PROMPT
                                 })

In [ ]:
query = "What ia the policy for theft"
qa.run(query)

In [ ]:
for qna in questions_answers:
    user_answer = "Yes it is"  # Replace this with the actual user's answer
    query = f"{qna['Is working off the clock permitted for non-exempt employees?']}: {qna[' No, working off the clock is never permitted for non-exempt employees.']} {user_answer}"

    expanded_answer = qa.run(query)  # Use the qa chain to process the query

    print(f"Question: {qna['question']}")
    print(f"Answer: {qna['answer']}")
    print(f"User Answer: {user_answer}")
    print(f"Expanded Answer: {expanded_answer}")
    print ("\n\n")


KeyError: ignored

In [ ]:
 You are an expert in comparing user answers to the correctly defined answers."


In [ ]:
\n\nQ: {question}\nA: {answer}\nUser Answer: {user_answer}
